In [ ]:
#!/bin/python
import glob
import pandas as pd
import numpy as np
import os
import torch
# import cPickle
import pickle
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster.k_means_ import KMeans
import sys
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
import re
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
from lightgbm import LGBMClassifier
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
#Usage: {0} kmeans_model, cluster_num, file_list
n_clusters = 50
model_path = 'knn_cnn_{}_model'.format(n_clusters)
video_list = './list/train.video'

kmeans_model = './kmeans/'+model_path+'.sav'; file_list = video_list
cluster_num = int(n_clusters)

In [ ]:
model = pickle.load(open(kmeans_model,"rb"))
model

In [ ]:
path = './cnn/*.pkl'

filelist = []

for file in glob.glob(path):
    filelist.append(file)

In [ ]:
def get_features(k, model, path_list, n_frames = 500):
    loaded_model = model
    k = k
    start_time = time.time()
# sample_dict = dict()
    features_list = []
    start_time = time.time()
    for i in range(len(filelist)):
    # for i in range(2500, 20000):
        with open(filelist[i], 'rb') as f:
            try:
                data = pickle.load(f)
                if len(data) != 0:
                    list_foo = []
                    foo = pickle.load(open(filelist[i],"rb"))
    #                 if len(foo) >= 20:
    #                     srs_idx = np.random.choice(len(foo), 10) # simple random sampling
    #                 else:
    #                     srs_idx = list(range(len(foo)))
                    for idx in range(len(foo)):
    #                 for idx in srs_idx:
                        list_foo.append(foo[idx].detach().numpy())
                    foo_concat = np.concatenate(list_foo, axis = 0)
                else:
                    foo_concat = np.zeros((1,1280))
            except EOFError:
                pass
        pred_centers = loaded_model.predict(foo_concat)
        num_clusters = k
        bow_preds = np.zeros((1, num_clusters))

        for ind in pred_centers:
            bow_preds[0, ind] += 1
        norm_feat = (1.0 * bow_preds)/np.sum(bow_preds)    
        features_list.append(norm_feat)    

        if i % 500 == 0:
            print('========== {}th step is completed! =========='.format(i))
        else:
            pass
    print('========== All complete! It takes {} secs. =========='.format(time.time() - start_time))    
    return features_list

In [ ]:
# foo = get_features(100, model, filelist, n_frames = 100)
foo = get_features(50, model, filelist, n_frames = 100)

In [ ]:
# # with open(filelist[0], 'rb') as f:
# #     data = pickle.load(f)
# foo = get_features(100, model, filelist, n_frames = 100)
data_array = np.concatenate(foo, axis = 0)
data_mart = pd.DataFrame(data_array)
data_mart.to_csv('./total_cnn_features_k{}.csv'.format(n_clusters), index=False)

In [ ]:
total_features = pd.read_csv('./total_cnn_features_k{}.csv'.format(n_clusters))
total_features.head(3)

In [ ]:
video_name_ind = []
for i in range(len(filelist)):
    match_front = re.search('cnn/', filelist[i])
    match_end = re.search('.pkl', filelist[i])
    video_name_ind.append(filelist[i][match_front.end():match_end.start()])
    video_name = pd.DataFrame({'video': video_name_ind})  

In [ ]:
k = n_clusters
column_name = ['video']
for i in range(k):
    column_name.append('feature_{}'.format(i))

In [ ]:
total_data = pd.concat([video_name, total_features], axis = 1)
total_data.columns = column_name
train_ind = pd.read_csv('./list/train', sep = ' ', header = None)
valid_ind = pd.read_csv('./list/val', sep = ' ', header = None)
test_ind = pd.read_csv('./list/test.video', sep = ' ', header = None)

In [ ]:
train_ind['Data'] = 'TRAIN'
valid_ind['Data'] = 'VALID'
test_ind[1] = 'UNK'
test_ind['Data'] = 'TEST'

In [ ]:
train_ind.columns = ['video','target','Data']
valid_ind.columns = ['video','target','Data']
test_ind.columns = ['video','target','Data']
data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)

In [ ]:
# data_lable['target_p001'] = 
data_lable['target_p001'] = data_lable['target']
data_lable['target_p002'] = data_lable['target']
data_lable['target_p003'] = data_lable['target']
data_lable['target_p001_10'] = 1
data_lable['target_p002_10'] = 1
data_lable['target_p003_10'] = 1

data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

In [ ]:
total_mart = total_data.merge(data_lable, how = 'right', on = 'video')
total_mart = total_mart.fillna(0)

train_mart = total_mart[total_mart['Data'] == 'TRAIN']
valid_mart = total_mart[total_mart['Data'] == 'VALID']
test_mart  = total_mart[total_mart['Data'] == 'TEST']

total_mart.to_csv('./datamart_cnn_total_k{}.csv'.format(n_clusters), index=False)
train_mart.to_csv('./datamart_cnn_train_k{}.csv'.format(n_clusters), index=False)
valid_mart.to_csv('./datamart_cnn_valid_k{}.csv'.format(n_clusters), index=False)
test_mart.to_csv('./datamart_cnn_test_k{}.csv'.format(n_clusters), index=False)

In [ ]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs

def modeling_ap_lgbm(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = LGBMClassifier(random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of LightGBM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs

In [ ]:
#Choose for each model based on MAP
Xgb_results_p001, _ = modeling_ap_xgb(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002, _ = modeling_ap_xgb(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
LGBM_results_p003, _ = modeling_ap_lgbm(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
SVM_results_p001 = modeling_ap_SVM(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002 = modeling_ap_SVM(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
SVM_results_p003 = modeling_ap_SVM(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
AdaB_results_p001, _ = modeling_ap_AdaB(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
AdaB_results_p002, _ = modeling_ap_AdaB(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
AdaB_results_p003, _ = modeling_ap_AdaB(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
Boost_results_p001, _ = modeling_ap_Boost(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Boost_results_p002, _ = modeling_ap_Boost(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Boost_results_p003, _ = modeling_ap_Boost(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
Xgb_results_p001, _ = modeling_ap_xgb(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002, _ = modeling_ap_xgb(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003, _ = modeling_ap_xgb(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
LGBM_results_p001, _ = modeling_ap_lgbm(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
LGBM_results_p002, _ = modeling_ap_lgbm(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
LGBM_results_p003, _ = modeling_ap_lgbm(k=n_clusters, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')